In [8]:
# https://machinelearningmastery.com/how-to-implement-major-architecture-innovations-for-convolutional-neural-networks/

# example of creating a CNN with an inception module
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Activation
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import add
import numpy as np
import pandas as pd

In [2]:
import pickle5 as pickle
with open( "/mnt/vdb/thesis/jax/AMPNonAMP.V3recal_nr.final.reps", 'rb') as file:
    AMPs_df = pickle.load(file)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(np.array(AMPs_df['reps'].to_list()), np.array(AMPs_df['class'].to_list()), test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
print(X_train.shape)
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
print(X_test.shape)
X_val = np.reshape(X_val,(X_val.shape[0],X_val.shape[1],1))
print(X_val.shape)

(135855, 1900, 1)
(45286, 1900, 1)
(45285, 1900, 1)


In [16]:
# function for creating an identity or projection residual module
def residual_module(layer_in, n_filters):
    merge_input = layer_in
     # check if the number of filters needs to be increase, assumes channels last format
    if layer_in.shape[-1] != n_filters:
        merge_input = Conv1D(n_filters, 1, padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
    # conv1
    conv1 = Conv1D(n_filters, 2, padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
    # conv2
    conv2 = Conv1D(n_filters, 3, padding='same', activation='linear', kernel_initializer='he_normal')(conv1)
    # add filters, assumes filters/channels last
    layer_out = add([conv2, merge_input])
    # activation function
    layer_out = Activation('relu')(layer_out)
    
    return layer_out

# function for creating a naive inception block
def naive_inception_module(layer_in, f1, f2, f3):
    # 1x1 conv
    conv1 = Conv1D(f1, 3, padding='same', activation='relu')(layer_in)
    # 3x3 conv
    conv3 = Conv1D(f2, 5, padding='same', activation='relu')(layer_in)
    # 5x5 conv
    conv5 = Conv1D(f3, 7, padding='same', activation='relu')(layer_in)
    # 3x3 max pooling
    pool = MaxPooling1D(3, strides=1, padding='same')(layer_in)
    # concatenate filters, assumes filters/channels last
    layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
    
    return layer_out
 
# define model input
visible = Input(shape=(1900,1))
# add inception module
incept = naive_inception_module(visible, 128, 256, 256)
print(incept)
# add resnet module
rest = residual_module(incept.shape, 128)

layer_out = add([incept ,rest])

layer = Flatten()(layer_out)
# create model
layer_4 = Dense(1211, activation='relu')(layer)
x_output = Dense(1, activation='sigmoid', kernel_regularizer=l2(0.0001))(layer_4)

model = Model(inputs=visible, outputs=x_output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# summarize model
model.summary()
# plot model architecture
plot_model(model, show_shapes=True)

Tensor("concatenate_10/concat:0", shape=(None, 1900, 641), dtype=float32)


AttributeError: 'TensorShape' object has no attribute 'shape'

In [5]:
def lr_schedule(epoch):
    
    lr = 1e-3
    if epoch > 80:
        lr = 0.1e-5
    elif epoch > 50:    
        lr = 0.3e-5
    elif epoch > 20:
        lr = 1e-4
        
    print(' Learning rate: ', lr)    
    return lr 

es = EarlyStopping(monitor='val_loss', patience=8, verbose=1)
# val_loss

checkpoint = ModelCheckpoint("/mnt/vdb/thesis/Inception.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', period=1)

# val_accuracy

lr_scheduler = LearningRateScheduler(lr_schedule)

history2 = model.fit(
    X_train, y_train,
    epochs=100, batch_size=128,
    validation_data=(X_val, y_val),
    callbacks=[ es,lr_scheduler]
    )


 Learning rate:  0.001
Epoch 1/100
1062/1062 [==============================] - 270s 255ms/step - loss: 4.3354e-06 - accuracy: 0.5036 - val_loss: 6.0075e-08 - val_accuracy: 0.4961
 Learning rate:  0.001
Epoch 2/100
 150/1062 [===>..........................] - ETA: 3:32 - loss: 6.0039e-08 - accuracy: 0.4964

KeyboardInterrupt: 

In [ ]:
y_probas = model.predict(X_test)
threshold = 0.5
y_predict = np.where(y_probas > threshold, 1, 0)
print(classification_report(y_test, y_predict))